## Run Fooocus

In [ ]:
!pip install pygit2==1.15.1
%cd /content
!git clone https://github.com/lllyasviel/Fooocus.git
%cd /content/Fooocus
!python entry_with_update.py --share --always-high-vram


## Download checkpoints and LoRAs from civitai
to download checkpoints and LoRAs from civitai you have to run Fooocus first

https://civitai.com/models

In [ ]:
# put the link of the model you want to download (checkpoint or LoRA)
# link can specify a version id and if not specified the latest version will be downloaded
# url example without version id: https://civitai.com/models/133005/juggernaut-xl
# url example with version id: https://civitai.com/models/133005?modelVersionId=252902

civitai_url = "https://civitai.com/models/133005/juggernaut-xl"

# -----------------------------
import requests
from urllib.parse import urlparse
from urllib.parse import parse_qs

allowed_types = ['Checkpoint', 'LORA']
save_locations = {
    'Checkpoint': '/content/Fooocus/models/checkpoints',
    'LORA': '/content/Fooocus/models/loras'
}

parsed_url = urlparse(civitai_url)

model_id = parsed_url.path.split('/')[parsed_url.path.split('/').index('models') + 1]
model_version_id = parse_qs(parsed_url.query).get('modelVersionId')

url = "https://civitai.com/api/v1/models/" + model_id
response = requests.get(url)

if response.status_code != 200:
  raise RuntimeError('model not found')

data = response.json()
model_type = data.get('type')

if model_type not in allowed_types:
  raise RuntimeError('model is not a checkpoint or LoRA')

model_versions = data.get('modelVersions')

selected_version = None

if model_version_id:
  for model_version in model_versions:
    if str(model_version.get('id')) == model_version_id[0]:
      selected_version = model_version
else:
  selected_version = model_versions[0]

if selected_version is None:
  raise RuntimeError("this version doesn't exist")

if "SDXL" not in selected_version.get('baseModel'):
  raise RuntimeError("this model is not SDXL")

files = selected_version.get('files')
primary_file = None

for f in files:
  if f.get('primary'):
    primary_file = f

download_url = primary_file.get('downloadUrl')
file_name = primary_file.get('name')

LOCATION = save_locations[model_type]
%cd $LOCATION

model_name = data.get('name')
selected_version_name = selected_version.get('name')
print(f'downloading {model_name} ({model_type} version {selected_version_name})')

get_ipython().system(f'wget -O "{file_name}" "{download_url}"')